In [5]:
select_example = input('Path or Example:')
FILE_PATH = None
SAMPLES = {
    1: 'KOAX_20240521_1844',
    2: 'KOAX_20240521_1849',
    3: 'KOAX_20240523_2138',

}
if select_example.isdigit():
    FILE_PATH = f"{'./Samples/'}{SAMPLES[int(select_example)]}" # I don't care if it is out of range, I just want to test the code
else:
    FILE_PATH = select_example
import os
fsize = os.stat(FILE_PATH).st_size
f = open(FILE_PATH, 'rb')
import bz2
import namedstruct as ns
import logging

In [6]:
f.seek(0)

log = logging.getLogger(__name__)

VHR = ns.nstruct(
    (ns.char[6],'Tape'),            # Magic number: AR2V00
    (ns.char[2],'Version'),         # Version number
    (ns.char[1],'End of Tape'),     # End of Tape marker
    (ns.char[3],'Sequence Number'), # Extension number
    (ns.uint32,'Date_d'),           # Date of Volume, days since 1/1/1970
    (ns.uint32,'Time_ms'),          # Time of Volume, milliseconds since midnight
    (ns.char[4],'Station'),         # Station ID (ICAO)
    name='VHR',padding=1,endian='>')
LDM_CR = ns.nstruct(
    (ns.int32,'ctrl_word'),      # Control word - lenth of the compressed data
    (ns.raw,'data'),                       # This data is bz2 compressed when saved
    name='LDM_CR',padding=1,endian='>') # TODO: Need to tell it to compress when packing, save the size to the control word


L2A = ns.nstruct(
    (VHR,'VHR'),                       # Volume Header Record
    (LDM_CR[0],'records'),             # LDM_CR records 
    name='L2A',padding=1,endian='>')

# test = VHR.parse(f.read(24))
fi, __ = L2A.parse(f.read())
# go back to the first LDM_CR
f.seek(24)
while(f.tell() < fsize): # read each LDM_CR into a record and add it to the archive - we do this since the datatype is raw here.
    l = f.tell()
    s = int.from_bytes(f.read(4))
    if(s==b'\xff\xff'):
        break
    f.seek(l)
    d = f.read(s+4)
    if (d == b''):
        log.debug('bad data')
    ldm = LDM_CR.create(d)
    fi.records.append(ldm)


In [7]:

if True:
    f.close() # close the file
    log.setLevel(0)

msg_header_channel_enum = ns.enum('msg_header_channel_enum', None, ns.uint8, bitwise=False,
                    # Note: if both redundent channels are 0s, single channel mode. 
                           RC_1 = 0,    # redundant channel 1
                           RC_2 = 1,    # redundant channel 2
                           unused = 2,  # unused. 
                           ORDA = 3     # ORDA channel (vs legacy)
                           )
# Message header structure 
msg_header = ns.nstruct(
        (ns.uint32[3],'prepad'),                # padding
        (ns.uint16,'message_size'),             # size of message in halfwords
        (msg_header_channel_enum,'channel'),    # actually bit field
        (ns.uint8,'message_type'),              # type of message
        (ns.uint16,'sequence'),                 # sequence number of message
        (ns.uint16,'date'),                     # days since 1/1/1970
        (ns.uint32,'milliseconds'),             # milliseconds since midnight
        # Remember, is these 2 are not 1, then the message is extended and needs to be combined with the next message
        (ns.uint16,'segments'),                 # number of segments in message  
        (ns.uint16,'segment'),                  # segment number of this message
        name='msg_header', padding=1, prepack=ns.packvalue(2432,'message_size'), endian='>') # TODO: Use size of message to determine size of message 

# Base message structure
msg_base = ns.nstruct(
    (msg_header,'header'),
    (ns.raw,'data'),
    name='msg_base',padding=1,endian='>')

# UNCOMPRESSED RECORD data
LDM_UR = ns.nstruct(
    (msg_base[0],'msg'),                # Messages - not sure that this will work since it will hold extended data
    name='LDM_UR',padding=1,endian='>') # TODO: Need to tell it to compress when packing, save the size to the control word

RECORD_HOLDER = ns.nstruct(
    (LDM_UR[0],'ldm'),
    name='RECORD_HOLDER',padding=1,endian='>')




In [8]:
CODE_1 = ns.enum('CODE_1',None, ns.uint8, bitwise=False)
CODE_2 = ns.enum('CODE_1',None, ns.uint16, bitwise=False)
SI_1 = ns.nstruct((ns.uint8,'si'),name='SI_1',padding=1,endian='>')
SI_2 = ns.nstruct((ns.uint16,'si'),name='SI_2',padding=1,endian='>')
SI_4 = ns.nstruct((ns.uint32,'si'),name='SI_4',padding=1,endian='>')
SSI_2 = ns.nstruct((ns.int16,'ssi'),name='sSI_2',padding=1,endian='>')
SSI_4 = ns.nstruct((ns.int16,'ssi'),name='sSI_4',padding=1,endian='>')

msg_31_table = ns.nstruct(
    (ns.raw,'data'), # data
    name='msg_31_table',padding=1,endian='>')

db_hdr = ns.nstruct(      # 33 - MSG 31 - Data Block Header
    (ns.char,'db_type'),
    (ns.char[3],'dm_type'),
    (ns.uint16,'db_size'), # size of the data block (bytes) / reserved
    name='db_hdr',padding=1,endian='>')

t_17a = ns.nstruct(      # 17A - MSG 31 - Data Header Block
    (ns.char[4],'ICAO'), # ICAO of the station
    (ns.uint32,'time_ms'), # time of the observation
    (ns.uint16,'date_d'), # date of the observation
    (ns.uint16,'az_n'), # azimuth # of the observation
    (ns.single,'az_a'), # azimuth angle of the observation (32 bit)
    (CODE_1,'compression'), # compression type, values can be 0-3
    (ns.uint8,'spare_17'), # spare
    (ns.uint16,'rad_len'), # length of the radial data (uncompressed) ubckydibg the header, 9360-14296
    (CODE_1,'az_res_spacing'), # azimuth resolution spacing, 1: 0.5, 2: 1.0
    (CODE_1, 'rad_stat'), # radial status 0-132
    (ns.uint8,'el_n'), # elevation number
    (ns.single,'el_a'), # elevation angle
    (CODE_1, 'rad_spot_blacking'), # radial spot blanking: 0=none, 1=radical, 2=elevation, 4=volume
    (SI_1,'az_index_mode'), #0: none, 1-100-> 0.001 to 1.00 deg
    (ns.uint16,'data_block_count'), # number of data blocks
    (ns.uint32,'dbp_vol_dc'), # data block pointer volume data constant     17E
    (ns.uint32,'dbp_el_dc'), # data block pointer elevation data constant   17F
    (ns.uint32,'dbp_rad_dc'), # data block pointer radial data constant     17H
    (ns.uint32,'dbp_ref_m'), # data block pointer for REF moments       17B, 17I
    (ns.uint32,'dbp_vel_m'), # data block pointer for VEL moments       17B, 17I
    (ns.uint32,'dbp_sw_m'), # data block pointer for SW moments         17B, 17I
    (ns.uint32,'dbp_zdr_m'), # data block pointer for ZDR moments       17B, 17I
    (ns.uint32,'dbp_phi_m'), # data block pointer for PHI moments       17B, 17I
    (ns.uint32,'dbp_rho_m'), # data block pointer for RHO moments       17B, 17I
    (ns.uint32,'dbp_cfp_m'), # data block pointer for CFP moments       17B, 17I
    name='t_17a',padding=1,endian='>')

t_17e = ns.nstruct(      # 17E - MSG 31 - Volume Data Constants
    (db_hdr,'header'),
    (ns.uint8,'v_major'), 
    (ns.uint8,'v_minor'), 
    (ns.single,'lat'), # latitude of the radar
    (ns.single,'lon'), # longitude of the radar
    (SSI_2,'height'), # height of the radar
    (ns.uint16,'fh_h'), # height of the feedhorn
    (ns.single,'r_cal_c'), # calibration constant
    (ns.single,'shv_h_tx'), # horizontal tx power
    (ns.single,'shv_v_tx'), # vertical tx power
    (ns.single,'sdr_cal'), # calibration of zdr
    (ns.single,'dp_i'), # inital dp 
    (ns.uint16,'vcp'), # volume coverage pattern
    (CODE_2,'proc_stat_28'), # processing status options - says uint16, but it is bitwise so
    (ns.uint16,'zdr_bewm'), # zdr bias estimate weighted mean
    (ns.uint8[6],'spare_padding'), # padding - 46-51
    name='t_17e',padding=1,endian='>',base=msg_31_table)

t_17f = ns.nstruct(      # 17F - MSG 31- Elevation Data Constants
    (db_hdr,'header'),
    (SSI_2,'atmos'), # atmospheric attenuation
    (ns.single,'cal_z'), # calibration constant - scaling for signal processor
    name='t_17f',padding=1,endian='>',base=msg_31_table)

t_17h = ns.nstruct(      # 17H - MSG 31 - Radical Data Constants
    (db_hdr,'header'),
    (SI_2,'unabm_r_int'), # unambiguous range interval size
    (ns.single,'noise_h'), # noise level for horizontal
    (ns.single,'noise_v'), # noise level for vertical
    (SI_2, 'nyq'), # nyquist velocity
    (ns.uint16,'rad_flags'), # radial flags
    (ns.single,'cal_const_h'), # calibration constant for horizontal
    (ns.single,'cal_const_v'), # calibration constant for vertical
    name='t_17h',padding=1,endian='>',base=msg_31_table)

t_17i = ns.nstruct(      # 17I - TODO
    (ns.raw,'data'), # data
    name='t_17i',padding=1,endian='>', base=msg_31_table)

t_17i_ref = ns.nstruct(      # 17I - MSG 31 - REF Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_ref',padding=1,endian='>')

t_17i_vel = ns.nstruct(      # 17I - MSG 31 - VEL Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_vel',padding=1,endian='>')

t_17i_sw = ns.nstruct(      # 17I - MSG 31 - SW Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_sw',padding=1,endian='>')

t_17i_zdr = ns.nstruct(      # 17I - MSG 31 - ZDR Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_zdr',padding=1,endian='>')

t_17i_phi = ns.nstruct(      # 17I - MSG 31 - DP Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_phi',padding=1,endian='>')

t_17i_rho = ns.nstruct(      # 17I - MSG 31 - CC Moments
    (ns.raw,'data'), # datarho
    base=t_17i,name='t_17i_rho',padding=1,endian='>')

t_17i_cfp = ns.nstruct(      # 17I - MSG 31 - Clutter Filter Pwoeer Removed - Moments
    (ns.raw,'data'), # data
    base=t_17i,name='t_17i_cfp',padding=1,endian='>')

t_17b = ns.nstruct(      # 17B - MSG 31 - Data Block
    (db_hdr,'header'),
    (ns.uint8,'reserved_a'), 
    (ns.uint8,'reserved_b'), 
    (ns.uint16,'n_gates'), # number of gates
    (SI_2,'gate_range'), # gate range to center of first gate - 0.000 to 32.768 km
    (SI_2,'gate_samp_int'), # gate sample interval - 0.25 to 4.0 km
    (SI_2,'tover'), # threshold value for similar to resolution, 0.0 to 20.0 dB
    (SI_2,'SNR_thres'), # signal to noise threshold for valid data, -12.0 to 20.0 dB
    (CODE_1,'ctrl_flags'), # control flags for recombined - 0=none, 1=az rads, 2=range gates, 3=rads and gates to legacy res
    (ns.uint8,'dw_size'), # data word size, 8 or 16
    (ns.single,'scale'), # scale value for data momens int->float
    (ns.single,'offset'), # offset value for data moments int->float
    (t_17i,'data moments'), # data moments                            17I       
    name='t_17b',padding=1,endian='>',base=msg_31_table)

msg_31 = ns.nstruct(       # 17A
    (msg_header,'header'),
    (t_17a,'header_31'), # data
    (msg_31_table[0],'data'), # data
    base=msg_base,name='msg_31',padding=1,endian='>')


def create_msg(data):
    r"""
    Create a message of the correct type from the data
    :param data: raw data
    """
    # tpyes to care about right now: {15: 5, 0: 122, 18: 4, 3: 1, 5: 1, 2: 3, 31: 10898}
    type = int.from_bytes(data[15:16], byteorder='big')
    message = None
    if(type==0):
        message = msg_base.create(data)
    elif(type==31):
        message = msg_31.create(data) # TODO: need to parse the data at the pointers
        if(message is None):
            print('bad data')
    else:
        message = msg_base.create(data)
    #TODO: parse the message into the correct message type instead of raw data

    return message

In [9]:
# log.setLevel(logging.FATAL-1)
record_count = len(fi.records)
print(f"Number of records: {record_count}")

record = 0
records = RECORD_HOLDER.create('')
for cr in fi.records:
    if(cr.ctrl_word < 0):
        break
    data = bz2.decompress(cr.data[:cr.ctrl_word])
    if data == b'':
        log.error(f"Error decompressing data. CW: {cr.ctrl_word}")
        break


    position = 0 # position in the data
    i = 0 # message counter
    ldm = LDM_UR.create('')
    while(position<len(data)):
        msg_size = int.from_bytes(data[position+12:position+14]) # get size in halfwords
        if(msg_size > 65535):
            log.error(f"Error: Message size too large: {msg_size}")
            raise Exception(f"Stop {msg_size}")
            break
        elif(msg_size<9):
            if (msg_size != 0):
                log.warning(f"Error?: Message size too small: {msg_size}")
                raise Exception(f"Stop {record}.{i}")
            else:
                log.info(f"Message {i} is empty")
                # break
        else:
            pass

        if(record == 0):
            eom = 2432*(i+1)-12 # not really needed but..
        else:
            eom = position+msg_size*2
        
        dat = data[position:eom]
        if dat[0:12] != b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00':
            log.error(f"Error: Prepad {i} is not 0: {dat[0:12]}")
        elif (dat[12:14] == b'\x00\x00' or dat[12:14] == b''):
            log.info(f"in {i}: Product is 0s: {dat[0:28]}")
        else:
            pass
        try:
            
            a_record_that_exists = create_msg(dat)
            ldm.msg.append(a_record_that_exists)
        except Exception as e:
            log.error(f"Error: {e} -  {dat[0:28]}")
            break
        position=eom+12
        i+=1
    records.ldm.append(ldm)
    record+=1
    # log.fatal(f"going to next record {record}")



Number of records: 94


Error: Prepad 105 is not 0: b''
Error: Cannot parse struct: data is corrupted. -  b''
Error: Prepad 14 is not 0: b''
Error: Cannot parse struct: data is corrupted. -  b''


In [10]:
def checkMsg(msg,i,j):
    warns = 0
    infos = 0
    if (msg is None):
        log.error(f"Error: Message {i}.{j} is None")
        return -1
    if msg.header.message_type > 33:
        log.warning(f"Message {i}.{j} type {msg.header.message_type} is not valid!")
        warns+=1
    if msg.header.message_type == 0:
        log.info(f"Message {i}.{j} has type 0. Likely a type 13 legacy area.")
        infos+=1
    if msg.header.milliseconds >= 86400000:
        log.warning(f"Message {i}.{j} time is invalid: {msg.header.milliseconds}")
        warns+=1
    if msg.header.segments > 1 or msg.header.segment > 1:
        log.warning(f"Message {i}.{j} has segments: {msg.header.segment} of {msg.header.segments}")
        warns+=1
    if msg.header.segments == 0 or msg.header.segment == 0:
        log.info(f"Message {i}.{j} has no segments: {msg.header.segment} of {msg.header.segments}")
        infos+=1
    if msg.header.date > 65535:
        log.warning(f"Message {i}.{j} date is invalid: {msg.header.date}")
        warns+=1
    if msg.header.sequence > 65535:
        log.warning(f"Message {i}.{j} sequence is invalid: {msg.header.sequence}")
        warns+=1
    if msg.header.sequence == 0:
        log.info(f"Message {i}.{j} has sequence number 0")
        infos+=1
    if msg.header.channel > 10:
        log.warning(f"Message {i}.{j} redundant channel is invalid: {msg.header.channel}")
        warns+=1
    if msg.header.message_size > 65535:
        log.warning(f"Message {i}.{j} size is invalid: {msg.header.message_size}")
        warns+=1
    if msg.header.prepad[0] != 0 or msg.header.prepad[1] != 0 or msg.header.prepad[2] != 0:
        log.warning(f"Message {i}.{j} Prepad is not 0: {msg.header.prepad}")
        warns+=1
    if(warns > 0):
        log.warning(f"...Message {i}.{j} - W:{warns} - {msg.header.message_type} has sequence number {msg.header.sequence} and is {msg.header.message_size} halfwords")
    type = msg.header.message_type
    if(type is None):
        log.error(f"Error: Message {i}.{j} type is None")
    return type
    

In [11]:

# log.setLevel(1000)
prods = {}
totals = 0
for i, ldm in enumerate(records.ldm):
    for j, msg in enumerate(ldm.msg):
        try:
            type = checkMsg(msg,i,j)
            prods[type] = prods.get(type,0)+1
        except Exception as e:
            print(e)
        
        totals+=1
print(prods)
print(totals)
# 3.2.4.17

Message 0.0 has segments: 1 of 5
...Message 0.0 - W:1 - 15 has sequence number 35150 and is 1208 halfwords
Message 0.1 has segments: 2 of 5
...Message 0.1 - W:1 - 15 has sequence number 35150 and is 1208 halfwords
Message 0.2 has segments: 3 of 5
...Message 0.2 - W:1 - 15 has sequence number 35150 and is 1208 halfwords
Message 0.3 has segments: 4 of 5
...Message 0.3 - W:1 - 15 has sequence number 35150 and is 1208 halfwords
Message 0.4 has segments: 5 of 5
...Message 0.4 - W:1 - 15 has sequence number 35150 and is 611 halfwords
Message 0.126 has segments: 1 of 4
...Message 0.126 - W:1 - 18 has sequence number 36412 and is 1208 halfwords
Message 0.127 has segments: 2 of 4
...Message 0.127 - W:1 - 18 has sequence number 36412 and is 1208 halfwords
Message 0.128 has segments: 3 of 4
...Message 0.128 - W:1 - 18 has sequence number 36412 and is 1208 halfwords
Message 0.129 has segments: 4 of 4
...Message 0.129 - W:1 - 18 has sequence number 36412 and is 1142 halfwords


{15: 5, 0: 122, 18: 4, 3: 1, 5: 1, 2: 3, 31: 10917}
11053
